### A logistic regression forward pass 

In [39]:
import torch.nn.functional as F
import torch
from torch.autograd import grad

In [40]:
y= torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0] , requires_grad=True)

In [41]:
# compute in an output layer
z = x1*w1+b
a = torch.sigmoid(z)

In [42]:
a, z

(tensor([0.9183], grad_fn=<SigmoidBackward0>),
 tensor([2.4200], grad_fn=<AddBackward0>))

In [43]:
loss = F.binary_cross_entropy(a,y)

In [44]:
loss

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)

### Updating w1 and b

In [45]:
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

In [46]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


### Multilayer perceptron with two hidden layers

In [47]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs:int, num_output:int):
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layers
            torch.nn.Linear(num_inputs,30), 
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),

            # Output layer 
            torch.nn.Linear(20, num_output)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [48]:
# Instantiation of new NeuralNetwork model
model = NeuralNetwork(num_inputs=50, num_output=3)

In [49]:
# Printing our model summary 
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [51]:
# Checking for total number of trainable parameters of our model 
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad==True)
print("Total number of trainable parametres are: ", num_params)

Total number of trainable parametres are:  2213


In [69]:
# we can access the corresponding weight parameter matrix(of a specifique layer) as follows
print(model.layers[0].weight[0])

tensor([ 0.0140,  0.1141, -0.0555, -0.0537, -0.1367,  0.1325, -0.0204, -0.0267,
        -0.0182, -0.0141, -0.1084,  0.1293,  0.0982, -0.0130, -0.1166,  0.1030,
        -0.1035,  0.1308, -0.0041, -0.1371, -0.0775, -0.1317,  0.1077,  0.1308,
        -0.0525, -0.1021,  0.0685,  0.1336, -0.1189,  0.0956, -0.0956,  0.1397,
        -0.0626, -0.1314, -0.0395, -0.0528,  0.1146,  0.0751, -0.0991, -0.0744,
        -0.0441,  0.0133,  0.1055, -0.0314, -0.0122, -0.1392, -0.0051,  0.0971,
        -0.0836,  0.0288], grad_fn=<SelectBackward0>)


In [75]:
print(model.layers[0].bias)

Parameter containing:
tensor([ 0.0306, -0.1222,  0.0285,  0.0298, -0.0192,  0.0409,  0.0888, -0.0540,
        -0.0216, -0.0953, -0.0094,  0.0729,  0.1143, -0.1376,  0.1097,  0.0583,
        -0.0987, -0.0095, -0.0149,  0.0376, -0.1405, -0.0510, -0.0211,  0.1295,
         0.0778, -0.0690,  0.0241,  0.0044, -0.0577,  0.0354],
       requires_grad=True)


In [77]:
# Generate random training data 
X = torch.rand((1,50))
X

tensor([[0.3135, 0.9837, 0.5922, 0.3073, 0.3608, 0.7955, 0.0664, 0.6533, 0.0822,
         0.0200, 0.5744, 0.6328, 0.9363, 0.0790, 0.0242, 0.6049, 0.3515, 0.0839,
         0.2789, 0.1170, 0.4995, 0.0073, 0.6496, 0.7226, 0.5823, 0.5777, 0.4274,
         0.5367, 0.2473, 0.7602, 0.3540, 0.6743, 0.0882, 0.0440, 0.4770, 0.4211,
         0.7572, 0.4479, 0.7891, 0.2577, 0.6708, 0.5272, 0.9537, 0.1902, 0.6983,
         0.8500, 0.5506, 0.2846, 0.3462, 0.5554]])

In [79]:
# training the model (Execute forward pass of a model)
out = model(X)
out

tensor([[ 0.1796, -0.2231, -0.0342]], grad_fn=<AddmmBackward0>)

In [80]:
# This tells PyTorch that it doesn't need to keep track of the
# gradients, which can result in significant savings in memory and
# computation.

with torch.no_grad():
    out = model(X)
print(out)

tensor([[ 0.1796, -0.2231, -0.0342]])


In [85]:
# Using softmax fonction as the activation function 
with torch.no_grad():
    out =torch.softmax(model(X),dim=1)
print(out)

tensor([[0.4039, 0.2700, 0.3261]])
